In [1]:
import tensorflow as tf
import numpy as np
from utils import dataUtil

N = 10 # depend on the N previous periods
B = 1 # batch size

dateSelected, stockPrice  = dataUtil.getData()
logreturn = dataUtil.logReturn(stockPrice)
# return for N previous periods, input, shape: [-1,L,N]
logReturn_x = dataUtil.logReturnMatrix(logreturn, N)
# return for current period
logReturn_x0 = logreturn[N:]

extracting csv data from data/sp10/
['ORCL.csv', 'EBAY.csv', 'INTU.csv', 'INTC.csv', 'ADSK.csv', 'FB.csv', 'SYMC.csv', 'NFLX.csv', 'GOOGL.csv', 'AAPL.csv']


In [2]:
L = len(stockPrice[0]) # L stocks
logReturn_x_data = tf.reshape(logReturn_x, [-1,L,N,1])

In [3]:
x = tf.placeholder(tf.float32, shape=[None, L, N])
x_data = tf.reshape(x, [-1,L,N,1]) # start from one channel
y_ = tf.placeholder(tf.float32, shape=[None, L])
previousPortfolio = tf.placeholder(tf.float32, shape=[None, L]) # portfolio for last time step
previousReturn = tf.placeholder(tf.float32, shape=[None]) # return for last time step

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
# convolution and pooling
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def avg_pool_4x1(x):
    return tf.nn.avg_pool(x, ksize=[1, 1, 1, 1],
                        strides=[1, 1, 1, 1], padding='SAME')



In [5]:
# first convolution layer
W_conv1 = weight_variable([1, 3, 1, 10])
b_conv1 = bias_variable([10])

h_conv1 = tf.nn.relu(conv2d(x_data, W_conv1) + b_conv1)
h_pool1 = avg_pool_4x1(h_conv1)

In [6]:
# second convolutional layer
W_conv2 = weight_variable([1, 3, 10, 5])
b_conv2 = bias_variable([5])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = avg_pool_4x1(h_conv2)

In [7]:
# densely connected layer 1
wsize1 = int(h_pool2.shape[2] * h_pool2.shape[3])
wsize2 = 1
W_fc1 = weight_variable([wsize1, wsize2] )
prePort = tf.reshape(previousPortfolio,[-1,1])
W_fc12 = weight_variable([1, wsize2])

b_fc1 = bias_variable([wsize2])
h_pool2_flat = tf.reshape(h_pool2, [-1, wsize1])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + tf.matmul(prePort, W_fc12) + b_fc1)
print tf.shape(h_fc1)
h_fc1_score = tf.reshape(h_fc1, [-1, L])

Tensor("Shape:0", shape=(2,), dtype=int32)


In [8]:
# densely connected layer 2 
# readout layer
W_fc2 = weight_variable([L, L])
b_fc2 = bias_variable([L])

y_conv = tf.matmul(h_fc1_score, W_fc2) + b_fc2

In [9]:
currentPortfolio = tf.nn.softmax(logits=y_conv)

reward0 = np.multiply(currentPortfolio, y_)
reward = tf.reduce_sum(reward0, 1)

# calculate return including transaction cost

c = np.zeros(L+1) + 0.0001 # transaction cost coefficients

flag = 0
for j in xrange(L):
    tmp = currentPortfolio[:,j]-previousPortfolio[:,j]*tf.divide(tf.exp(x[:,j,-1]),(1.0+previousReturn))
    reward = reward - c[j+1] * np.abs(tmp)
    if tmp != 0:
        flag = 1
if flag:
    reward = reward - c[0]

# loss function: reward_minus
reward_minus = -tf.reduce_prod(reward+1)

In [10]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(reward_minus)
test_step = tf.train.AdamOptimizer(1e-4).minimize(reward_minus)

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    B = 1
    pre_train_accumulate = 1
    for _ in range(500):
        PrePortfolio = np.zeros([B,L])
        PreReturn = np.zeros(B)
        train_accumulate = 1
        for i in range(len(logReturn_x0)/2/B):
            historicalData = logReturn_x[i*B:(i+1)*B]
            compareReturn = logReturn_x0[i*B:(i+1)*B]
            currentReturn = sess.run(reward, {x: historicalData, y_: compareReturn, \
                                      previousPortfolio: PrePortfolio, previousReturn: PreReturn} )
            train_accumulate = train_accumulate * (sess.run(-reward_minus,\
                                                {x: historicalData,y_: compareReturn, \
                                                 previousPortfolio: PrePortfolio, previousReturn: PreReturn}))
            train_step.run(feed_dict={x: historicalData, y_: compareReturn,\
                                  previousPortfolio: PrePortfolio, previousReturn: PreReturn})
            prePortfolio = sess.run(currentPortfolio, \
                               {x: historicalData,y_: compareReturn, \
                                                 previousPortfolio: PrePortfolio, previousReturn: PreReturn})
            preReturn = currentReturn
        if np.abs(train_accumulate - pre_train_accumulate)<1e-8:
            break
        else:
            pre_train_accumulate = train_accumulate
        print train_accumulate
    test_accumulate = 1 
    PrePortfolio = np.zeros([B,L])
    PreReturn = np.zeros(B)
    testReturn = []
    for i in range(len(logReturn_x0)/2/B, len(logReturn_x0)/B):
        historicalData = logReturn_x[i*B:(i+1)*B]
        compareReturn = logReturn_x0[i*B:(i+1)*B]
        currentReturn = sess.run(reward, {x: historicalData, y_: compareReturn, \
                                      previousPortfolio: PrePortfolio, previousReturn: PreReturn} )
        testReturn.append(currentReturn)
        test_accumulate = test_accumulate * (sess.run(-reward_minus,\
                                                {x: historicalData,y_: compareReturn, \
                                                 previousPortfolio: PrePortfolio, previousReturn: PreReturn}))
        test_step.run(feed_dict={x: historicalData, y_: compareReturn,\
                                  previousPortfolio: PrePortfolio, previousReturn: PreReturn})
        prePortfolio = sess.run(currentPortfolio, \
                               {x: historicalData,y_: compareReturn, \
                                                 previousPortfolio: PrePortfolio, previousReturn: PreReturn})
        preReturn = currentReturn
     


1.08916359558
1.0898507924
1.09056510265
1.09141417651
1.09242524314
1.09362991616
1.09506023897
1.09675691194
1.09876825154
1.10115266591
1.10398519296
1.10735586483
1.11137107485
1.11615315294
1.12184090435
1.12858516249
1.1365480829
1.14591798947
1.15691446619
1.16977460221
1.18471769751
1.20190649339
1.22141214148
1.24319832787
1.26710886178
1.29285641102
1.32004199431
1.34819399659
1.37681205859
1.40542600503
1.43361711668
1.461051163
1.48747072613
1.51269557517
1.53660608854
1.55912569789
1.58021797404
1.59986591462
1.61808138257
1.63488847834
1.65032704865
1.66445102358
1.6773228189
1.6890144181
1.69960298403
1.70916830828
1.71779240328
1.72555646799
1.73253700058
1.73880966748
1.74444316282
1.74950329682
1.75405090712
1.75813613696
1.76181282543
1.76512303103
1.76810653041
1.7707972451
1.77322799793
1.77542813506
1.77742063754
1.77922859243
1.78087050415
1.78236340489
1.78372387426
1.78496474015
1.78610069205
1.78713736151
1.78808748341
1.78895875802
1.78975933407
1.79049574115

In [13]:
print train_accumulate, test_accumulate
testReturn_cnn = [float(testReturn[i]) for i in xrange(len(testReturn))]
print np.shape(testReturn_cnn)

print np.mean(testReturn_cnn)/np.std(testReturn_cnn)
print testReturn_cnn

1.80056320453 1.81968611733
(72,)
0.218189167
[-0.014886335469782352, -0.055797234177589417, -0.04151482135057449, -0.012503153644502163, 0.05783998966217041, 0.012669534422457218, -0.10510943084955215, 0.054019056260585785, 0.019063789397478104, 0.011736541986465454, 0.0054995957762002945, 0.012253908440470695, 0.01374413538724184, 0.026423875242471695, -0.0281561017036438, -0.004826952237635851, 0.0018615698209032416, 0.06360967457294464, -0.025831026956439018, 0.15862196683883667, 0.06556577980518341, -0.02797495573759079, -0.008308368735015392, -0.07586748152971268, 0.04141828045248985, 0.010269846767187119, 0.04480123519897461, -0.008660349994897842, 0.016663767397403717, 0.02760113775730133, -0.012515262700617313, 0.026278560981154442, 0.017219431698322296, 0.055203359574079514, 0.002638779114931822, 0.016441741958260536, -0.020243743434548378, 0.015309805050492287, -0.0007549322908744216, -0.009834209457039833, 0.01891256496310234, -0.011778581887483597, 0.020354025065898895, 0.

In [27]:
 print np.shape(range(len(logReturn_x0)/2/B, len(logReturn_x0)/B))


(72,)


In [109]:
print 1e-7

1e-07


In [12]:
a = [1,2,3,4]
print a-1

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [1]:
import tensorflow as tf
import numpy as np
from utils import dataUtil

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# convolution and pooling
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def avg_pool_4x1(x):
    return tf.nn.avg_pool(x, ksize=[1, 1, 1, 1],
                        strides=[1, 1, 1, 1], padding='SAME')


def CNN(stockPrice, Time, c):
    logreturn = dataUtil.logReturn(stockPrice)
    N = 10 # depend on the N previous periods
    # return for N previous periods, input, shape: [-1,L,N]
    logReturn_x = dataUtil.logReturnMatrix(logreturn, N)
    # return for current period
    logReturn_x0 = logreturn[N:]
    
    L = len(stockPrice[0]) # L stocks
    B = 1 # batch size
    
    # training and testing data
    Time = np.array(Time) - 1 - N
    TestIndex = Time
    TrainIndex = range(Time[0])
    train_x = [logReturn_x[i] for i in TrainIndex]
    train_y = [logReturn_x0[i] for i in TrainIndex]
    test_x = [logReturn_x[i] for i in TestIndex]
    test_y = [logReturn_x0[i] for i in TestIndex]
    
    # get CNN model
    
    # placeholder
    x = tf.placeholder(tf.float32, shape=[None, L, N])
    x_data = tf.reshape(x, [-1,L,N,1]) # start from one channel
    y_ = tf.placeholder(tf.float32, shape=[None, L])
    previousPortfolio = tf.placeholder(tf.float32, shape=[None, L]) # portfolio for last time step
    previousReturn = tf.placeholder(tf.float32, shape=[None]) # return for last time step
    
    # first convolution layer
    W_conv1 = weight_variable([1, 3, 1, 10])
    b_conv1 = bias_variable([10])
    h_conv1 = tf.nn.relu(conv2d(x_data, W_conv1) + b_conv1)
    h_pool1 = avg_pool_4x1(h_conv1)
    
    # second convolutional layer
    W_conv2 = weight_variable([1, 3, 10, 5])
    b_conv2 = bias_variable([5])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = avg_pool_4x1(h_conv2)

    # densely connected layer 1
    wsize1 = int(h_pool2.shape[2] * h_pool2.shape[3])
    wsize2 = 1
    W_fc1 = weight_variable([wsize1, wsize2] )
    prePort = tf.reshape(previousPortfolio,[-1,1])
    W_fc12 = weight_variable([1, wsize2])
    b_fc1 = bias_variable([wsize2])
    h_pool2_flat = tf.reshape(h_pool2, [-1, wsize1])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + tf.matmul(prePort, W_fc12) + b_fc1)
    h_fc1_score = tf.reshape(h_fc1, [-1, L])
    
    # densely connected layer 2 
    # readout layer
    W_fc2 = weight_variable([L, L])
    b_fc2 = bias_variable([L])
    y_conv = tf.matmul(h_fc1_score, W_fc2) + b_fc2
    
    # produce portfolio
    currentPortfolio = tf.nn.softmax(logits=y_conv)

    # define loss function
    reward0 = np.multiply(currentPortfolio, y_)
    reward = tf.reduce_sum(reward0, 1)
    # calculate return including transaction cost
    flag = 0
    for j in xrange(L):
        tmp = currentPortfolio[:,j]-previousPortfolio[:,j]*tf.divide(tf.exp(x[:,j,-1]),(1.0+previousReturn))
        reward = reward - c[j+1] * np.abs(tmp)
        if tmp != 0:
            flag = 1
    if flag:
        reward = reward - c[0]

    # loss function: reward_minus
    reward_minus = -tf.reduce_prod(reward+1)
    
    
    
    train_step = tf.train.AdamOptimizer(1e-4).minimize(reward_minus)
    test_step = tf.train.AdamOptimizer(1e-4).minimize(reward_minus)
    
    # train and test model
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        # train model
        pre_train_accumulate = 1
        for k in range(500):
            PrePortfolio = np.zeros([B,L])
            PreReturn = np.zeros(B)
            train_accumulate = 1
            for i in TrainIndex:
                historicalData = logReturn_x[i*B:(i+1)*B]
                compareReturn = logReturn_x0[i*B:(i+1)*B]
                currentReturn = sess.run(reward, {x: historicalData, y_: compareReturn, previousPortfolio: PrePortfolio, previousReturn: PreReturn} )
                train_accumulate = train_accumulate * (sess.run(-reward_minus,{x: historicalData,y_: compareReturn, previousPortfolio: PrePortfolio, previousReturn: PreReturn}))
                train_step.run(feed_dict={x: historicalData, y_: compareReturn,previousPortfolio: PrePortfolio, previousReturn: PreReturn})
                prePortfolio = sess.run(currentPortfolio, {x: historicalData,y_: compareReturn, previousPortfolio: PrePortfolio, previousReturn: PreReturn})
                preReturn = currentReturn
            if np.abs(train_accumulate - pre_train_accumulate)<1e-8:
                break
            else:
                pre_train_accumulate = train_accumulate
            print 'epoch{0}, the training accumulated return is {1}.'.format(k, train_accumulate)
            
        # test model
        test_accumulate = 1 
        PrePortfolio = np.zeros([B,L])
        PreReturn = np.zeros(B)
        testReturn = []
        for i in TestIndex:
            historicalData = logReturn_x[i*B:(i+1)*B]
            compareReturn = logReturn_x0[i*B:(i+1)*B]
            currentReturn = sess.run(reward, {x: historicalData, y_: compareReturn, previousPortfolio: PrePortfolio, previousReturn: PreReturn} )
            testReturn.append(currentReturn)
            test_accumulate = test_accumulate * (sess.run(-reward_minus, {x: historicalData,y_: compareReturn, previousPortfolio: PrePortfolio, previousReturn: PreReturn}))
            test_step.run(feed_dict={x: historicalData, y_: compareReturn, previousPortfolio: PrePortfolio, previousReturn: PreReturn})
            prePortfolio = sess.run(currentPortfolio,  {x: historicalData,y_: compareReturn,  previousPortfolio: PrePortfolio, previousReturn: PreReturn})
            preReturn = currentReturn
            
    return testReturn



# get data (date, stockPrice)
dateSelected, stockPrice = dataUtil.getData()
    
# get time for baseline estimation  
Time = range(10+(len(dateSelected)-10)/2+1,len(dateSelected)) 

# Date for estimated return period (startDate,endDate) =  (dateSelected[Time[i]-1],dateSelected[Time[i]])
Date = [(dateSelected[i-1][0],dateSelected[i][0]) for i in Time]

# parameters for transaction cost
c = np.zeros(len(stockPrice[-1])+1) + 0.0001

# estimated period return for corresponding date
estimateReturn = CNN(stockPrice, Time, c)
     


    



extracting csv data from data/sp10/
['ORCL.csv', 'EBAY.csv', 'INTU.csv', 'INTC.csv', 'ADSK.csv', 'FB.csv', 'SYMC.csv', 'NFLX.csv', 'GOOGL.csv', 'AAPL.csv']
epoch0, the training accumulated return is 1.09061932492.
epoch1, the training accumulated return is 1.09159474442.
epoch2, the training accumulated return is 1.09257390596.
epoch3, the training accumulated return is 1.09367822721.


KeyboardInterrupt: 